<a href="https://colab.research.google.com/github/ShrivatsaMishra/NLP_Final_Project/blob/main/RoBERTa_ELECTRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers[torch] accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 25.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [ ]:
!nvidia-smi

Mon Dec 11 17:02:44 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0    44W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
import json

def load_data(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            data.append(json.loads(line))
    return data

# Load data
dev_data = load_data('/content/drive/MyDrive/Task A/subtaskA_dev_monolingual.jsonl')
train_data = load_data('/content/drive/MyDrive/Task A/subtaskA_train_monolingual.jsonl')

In [ ]:
# from transformers import RobertaTokenizer

# tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

from transformers import ElectraTokenizer, ElectraForSequenceClassification, Trainer, TrainingArguments

# Load the tokenizer and model
tokenizer = ElectraTokenizer.from_pretrained('google/electra-base-discriminator')

def tokenize_data(data):
    return tokenizer([example['text'] for example in data], padding=True, truncation=True, max_length=512)

train_encodings = tokenize_data(train_data)
dev_encodings = tokenize_data(dev_data)

tokenizer_config.json:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

In [ ]:
import torch

# class TextDataset(torch.utils.data.Dataset):
class ElectraDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Prepare datasets
train_labels = [example['label'] for example in train_data]
dev_labels = [example['label'] for example in dev_data]

train_dataset = ElectraDataset(train_encodings, train_labels)
dev_dataset = ElectraDataset(dev_encodings, dev_labels)


In [ ]:
train_dataset

In [ ]:
# from transformers import RobertaForSequenceClassification
from transformers import ElectraTokenizer, ElectraForSequenceClassification, Trainer, TrainingArguments

model = ElectraForSequenceClassification.from_pretrained('google/electra-base-discriminator', num_labels=2)
# model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)
model.to(device)

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset
)

In [ ]:
trainer.train()

Step,Training Loss
500,0.207700
1000,0.094200
1500,0.048600
2000,0.039100
2500,0.032400
3000,0.042300
3500,0.019400
4000,0.021900
4500,0.022300
5000,0.013900


TrainOutput(global_step=7485, training_loss=0.039313202917217495, metrics={'train_runtime': 2747.251, 'train_samples_per_second': 43.592, 'train_steps_per_second': 2.725, 'total_flos': 3.150939065674752e+16, 'train_loss': 0.039313202917217495, 'epoch': 1.0})

In [ ]:
trainer.evaluate()


{'eval_loss': 2.292203426361084,
 'eval_runtime': 33.8693,
 'eval_samples_per_second': 147.626,
 'eval_steps_per_second': 2.332,
 'epoch': 1.0}

In [ ]:
torch.save(model, '/content/drive/MyDrive/Task A/RoBERTa1epoch')

In [ ]:
predictions = trainer.predict(dev_dataset)

In [ ]:
print(predictions)

PredictionOutput(predictions=array([[ 2.0531836, -2.128745 ],
       [-4.497766 ,  4.792103 ],
       [-2.0001485,  2.429315 ],
       ...,
       [ 5.1834655, -5.42246  ],
       [ 5.1962023, -5.428854 ],
       [ 5.104041 , -5.373665 ]], dtype=float32), label_ids=array([1, 1, 1, ..., 0, 0, 0]), metrics={'test_loss': 2.506685972213745, 'test_runtime': 34.1026, 'test_samples_per_second': 146.616, 'test_steps_per_second': 2.317})


In [ ]:
import numpy as np
from sklearn.metrics import f1_score

# Convert logits to binary predictions
pred_labels = np.argmax(predictions.predictions, axis=1)

# Calculate F1 score
f1 = f1_score(predictions.label_ids, pred_labels)
print(f"F1 Score: {f1}")

F1 Score: 0.5647461494580719


In [ ]:
# Make predictions on the training set
train_predictions = trainer.predict(train_dataset)

In [ ]:
# Convert logits to binary predictions
train_pred_labels = np.argmax(train_predictions.predictions, axis=1)

# Calculate F1 score
train_f1 = f1_score(train_predictions.label_ids, train_pred_labels)
print(f"Training F1 Score: {train_f1}")


Training F1 Score: 0.9987336727916759


In [ ]:
from transformers import RobertaForSequenceClassification

model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)
model.to(device)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,  # or however many you want
    per_device_train_batch_size=48,
    per_device_eval_batch_size=64,
    learning_rate=1e-5,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    evaluation_strategy="steps",  # or 'epoch' depending on your preference
    eval_steps=500,  # How often to perform evaluation within an epoch
    save_strategy="steps",  # Save at regular step intervals
    save_steps=500,  # Save every 500 steps
    load_best_model_at_end=True,  # Required for EarlyStoppingCallback
    metric_for_best_model='f1',  # Ensure
    greater_is_better=True
)

In [ ]:
from sklearn.metrics import f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average='binary')  # Adjust the average parameter as needed
    return {
        'f1': f1,
        # You can add other metrics here as well
    }


In [ ]:
from transformers import EarlyStoppingCallback
from transformers import Trainer, TrainingArguments

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=10)],  # Adjust the patience as needed
)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,F1
500,0.368100,0.649700,0.781395
1000,0.119200,0.809485,0.776810
1500,0.085200,0.486042,0.833130
2000,0.077500,0.745201,0.779980
2500,0.065100,0.899947,0.736471
3000,0.049200,0.870894,0.790140
3500,0.049400,0.645906,0.838987
4000,0.045000,0.551921,0.861122
4500,0.046400,0.784331,0.841101
5000,0.044500,0.802472,0.807610


TrainOutput(global_step=13500, training_loss=0.04485650980031049, metrics={'train_runtime': 14669.884, 'train_samples_per_second': 81.635, 'train_steps_per_second': 1.701, 'total_flos': 1.704920172074496e+17, 'train_loss': 0.04485650980031049, 'epoch': 5.41})

In [ ]:
# Make predictions on the training set
train_predictions = trainer.predict(train_dataset)

In [ ]:
import numpy as np
# Convert logits to binary predictions
train_pred_labels = np.argmax(train_predictions.predictions, axis=1)

# Calculate F1 score
train_f1 = f1_score(train_predictions.label_ids, train_pred_labels)
print(f"Training F1 Score: {train_f1}")

Training F1 Score: 0.9357600291999867


In [ ]:
dev_predictions = trainer.predict(dev_dataset)

In [ ]:
import numpy as np
from sklearn.metrics import f1_score

# Convert logits to binary predictions
pred_labels = np.argmax(dev_predictions.predictions, axis=1)

# Calculate F1 score
f1 = f1_score(dev_predictions.label_ids, pred_labels)
print(f"Dev F1 Score: {f1}")

Dev F1 Score: 0.8749266001174397


In [ ]:
torch.save(model, '/content/drive/MyDrive/Task A/ELECTRA')